In [2]:
import os.path

from revscoring import Model


model_path = os.path.expanduser("~/editquality/models/enwiki.damaging.gradient_boosting.model")
sm = Model.load(open(model_path), error_on_env_check=False)

Differences between the current environment and the environment in which the model was constructed environment were detected:
 - release '4.9.0-6-amd64' mismatch with original environment '4.16.0'
 - version '#1 SMP Debian 4.9.82-1+deb9u3 (2018-03-02)' mismatch with original environment '#1 SMP Wed Apr 18 14:02:11 PDT 2018'
 - platform 'Linux-4.9.0-6-amd64-x86_64-with-debian-9.4' mismatch with original environment 'Linux-4.16.0-x86_64-with-debian-9.4'
 - revscoring_version '2.2.2' mismatch with original environment '2.2.5'


In [4]:
import mwapi
from revscoring.extractors import api

extractor = api.Extractor(mwapi.Session("https://en.wikipedia.org", user_agent="ORES-LIME demo"))

In [9]:
rev_to_score = 12345678
feature_values = list(extractor.extract(rev_to_score, sm.features))
sm.score(feature_values)

{'prediction': False,
 'probability': {False: 0.9006672855880219, True: 0.09933271441197815}}

In [13]:
# FIXME: We would actually want to change the diff, though?
feature_values = list(extractor.extract(rev_to_score, sm.features, cache={"datasource.revision.text": "you"}))
sm.score(feature_values)

{'prediction': True,
 'probability': {False: 0.47502357204620316, True: 0.5249764279537968}}

In [ ]:
# TODO: explain both as tokens, and features.

from lime.lime_tabular import LimeTabularExplainer

# FIXME: Need to supply with training data
explainer = LimeTabularExplainer(
    None, # FIXME
    mode="classification",
    feature_names=sm.features,
    class_names=["damaging", "not damaging"],
    discretize_continuous=False
)
exp = explainer.explain_instance